In [56]:
import imgaug as ia
import os
import imageio
from imgaug.augmentables.polys import Polygon, PolygonsOnImage
import imgaug.augmenters as iaa
import numpy as np
import random
import tqdm
#Useful resources
#augment polygons
#https://nbviewer.org/github/aleju/imgaug-doc/blob/master/notebooks/B03%20-%20Augment%20Polygons.ipynb
#multicore cpu augment
#https://nbviewer.org/github/aleju/imgaug-doc/blob/master/notebooks/A03%20-%20Multicore%20Augmentation.ipynb

# provide dataset path with following structure
# data
#   images
#       train
#           *.png
#       val
#           *.png
#   labels
#       train
#           *.txt
#       val
#           *.txt

# Where format of label txt files is classId x1 y1 x2 y2 x3 y3 x4 y4

dataset_path = '../augmented_dataset'
images_path = os.path.join(dataset_path, 'images')
labels_path = os.path.join(dataset_path, 'labels')

In [7]:

import json
import shutil

def read_json_file(path_to_file: str):
    with open(path_to_file, "r") as p:
        return json.load(p)

def read_labelme_anno(path_anno):
    for dir_, _, files in os.walk(path_anno):
        for file_name in files:
            if not file_name[0] == '.':
                rel_dir = os.path.relpath(dir_, path_anno)
                rel_file = os.path.join(rel_dir, file_name)
                _labelme_anno = read_json_file(rel_file)
                print(_labelme_anno)
    # for _path in os.listdir(path_anno):
    #     _anno_rel_path = os.path.relpath(_path, start="/content/gdrive/MyDrive/Work/PolygonObjectDetection-main/data_rotation/labels/train")
    #     print(_anno_rel_path)
    #     _labelme_anno = read_json_file(_anno_rel_path)
    #     print(_labelme_anno)


catidx = {"tube": 0}

def convert_annos(labels_path):
    # root dir is cwd in most cases 
    # dataset_path is the name of the dataset
    # file structure:
    #   - root
    #       - dataset_path
    #           - train
    #               - images
    #               - labels
    #               - polygon_labels
    #           - val
    #               - images
    #               - labels
    #               - polygon_labels
    # then you can manually change the name of the folders if you want
    for dataset_folder in os.listdir(labels_path):
        if not dataset_folder[0] == '.':
            # train
            polygon_anno_path = os.path.join(os.path.dirname(labels_path), "polygon_labels", dataset_folder)
            if os.path.exists(polygon_anno_path):
                shutil.rmtree(polygon_anno_path)
            os.makedirs(polygon_anno_path)
            json_dataset_folder = os.path.join(labels_path, dataset_folder)
            for anno_file in os.listdir(json_dataset_folder):
                if not anno_file[0] == '.':
                    json_path = os.path.join(json_dataset_folder, anno_file)
                    new_json_path = os.path.join(polygon_anno_path, os.path.splitext(anno_file)[0] + ".txt")
                    anno = read_json_file(json_path)
                    annos_list = []
                    for polygon in anno['shapes']:
                        coordinates = []
                        for point in polygon['points']:
                            coordinates += point 
                        assert len(coordinates) == 8, "Label does not have four points in "+json_path
                        cat = catidx[polygon['label']]
                        labels = [cat, *coordinates]
                        label, label_pixel = normalize_anchors(labels, anno["imageHeight"], anno["imageWidth"])
                        annos_list.append(label)
                    annos_array = np.array(annos_list)
                    for an in annos_array:
                        if len(an)==11:
                            print(an)
                    # write to new_json_path
                    np.savetxt(str(new_json_path), annos_array, fmt=["%i"]+["%.6f"]*8)



# anno = convert_annos("/Users/antoniomorais/Work/PolygonObjectDetection/data_rotation/labels")
 

def get_image_annotations(label_json: str):
    # read json of labelme labels of an image, return list of polygons with classId
    polygon_list = []
    _labelme_anno = read_json_file(label_json)
    for polygon in _labelme_anno['shapes']:
        coordinates = []
        for point in polygon['points']:
            coordinates += point 
        assert len(coordinates) == 8, "Label does not have four points in "+label_json
        cat = catidx[polygon['label']]
        labels = [cat, *coordinates]
        polygon_list.append(labels)
    return polygon_list

In [38]:
# we want to read polygons for a single image (first)
# then we want to read every polygon of every image and store them in lists or something
# [ [ polygon1 polygon2 polygon3 ... ], [ polygon1 polygon2 polygon3 ... ], ...]
# this is the format for a polygon
# in our case we need to read labelme annotations and read here

# select set you want to augment here
images_set_path = os.path.join(images_path, 'val')
labels_set_path = os.path.join(labels_path, 'val')


json_path = os.path.join(labels_set_path, "Image__2021-12-13__11-26-09.json")

%matplotlib inline

# image = imageio.imread(os.path.join(images_set_path, "Image__2021-12-13__11-26-09.png"))
# ia.imshow(image)

# _polygon_list = []
# annotations_list = get_image_annotations(json_path)
# for polygon in annotations_list:
#     _, x1, y1, x2, y2, x3, y3, x4, y4 = polygon
#     # right now only works for 4 cornered polygon
#     _polygon = Polygon([
#         (x1, y1), 
#         (x2, y2),  
#         (x3, y3),  
#         (x4, y4)
#     ])
#     _polygon_list.append(_polygon)
    
# image_polys = np.copy(image)
# for _pol in _polygon_list:
#     image_polys = _pol.draw_on_image(image_polys, alpha_face=0.2, size_points=7)
# ia.imshow(image_polys)

images_list = []
polygons_list = []

# this is for doing augmentation on an entire set
for img in os.listdir(images_set_path):
    # if hidden file
    if img[0] == '.':
        continue
    img_path = os.path.join(images_set_path, img)
    image = imageio.imread(img_path)
    images_list.append((img_path, image))

    img_name = os.path.splitext(img)[0]
    json_path = os.path.join(labels_set_path, img_name + '.json')


    _poly_list = []
    annotations_list = get_image_annotations(json_path)
    for polygon in annotations_list:
        _, x1, y1, x2, y2, x3, y3, x4, y4 = polygon
        # right now only works for 4 cornered polygon
        _polygon = Polygon([
            (x1, y1), 
            (x2, y2),  
            (x3, y3),  
            (x4, y4)
        ])
        _poly_list.append(_polygon)
    polygons_list.append((json_path, _poly_list))

#images_list[0] = (path, img)
#polygons_list[0] = (path, [poly0, poly1, poly2, ...])


In [53]:

def normalize_anchors(label, img_h, img_w):
    """
        polygon
        FROM class id, x1, y1, x2, y2, x3, y3, x4, y4 (unnormalized)
        TO class id (unchanged), x1, y1, x2, y2, x3, y3, x4, y4 (normalized to [0, 1])
    """
    label = np.array(label)
    label_pixel = np.copy(label)
    label[1::2] = label[1::2]/img_w
    label[2::2] = label[2::2]/img_h
    # Common out the following lines to enable: polygon corners can be out of images
    # label[1::2] = label[1::2].clip(0., img_w)/img_w
    # label[2::2] = label[2::2].clip(0., img_h)/img_h
    # label_pixel[1::2] = label_pixel[1::2].clip(0., img_w)
    # label_pixel[2::2] = label_pixel[2::2].clip(0., img_h)
    return label, label_pixel

In [54]:



def augment_data(num_augmentations: int, path_augmentations: str):
    """
    num_augmentations: number of augmented images to generate
    path_augmentations: path to folder on which to put augmentations
    """
    count=0
    while os.path.exists(path_augmentations):
        count += 1 
    if count > 0:
        path_augmentations += str(count)
        print("Path already exists. Creating images in " + path_augmentations + "instead")
    
    augmented_images_folder_path = os.path.join(path_augmentations, "images")
    augmented_labels_folder_path = os.path.join(path_augmentations, "labels")

    os.makedirs(augmented_images_folder_path)
    os.makedirs(augmented_labels_folder_path)
    images_poly_aug = []
    # TODO specify number of augmentations here

    # TODO VERIFY THIS shuffle image and labels lists
        
    shuffled_list = list(zip(images_list, polygons_list))
    random.shuffle(shuffled_list)

    for idx, img_tuple, polygons_tuple in tqdm(enumerate(zip(*shuffled_list))):
        if idx == num_augmentations:
            return
        img_path, img = img_tuple
        polygons_path, polygons = polygons_tuple
        _polygons_on_image = ia.PolygonsOnImage(polygons, shape=image.shape)

        aug = iaa.Sequential([
            iaa.AdditiveGaussianNoise(scale=10),
            iaa.CoarseDropout(0.3, size_percent=0.005),
            iaa.AddToHueAndSaturation((-50, 50)),
            iaa.Affine(rotate=(-20, 20), translate_percent=(-0.3, 0.3), scale=(0.8, 1.2),
                mode=["constant", "edge"], cval=0),
            iaa.Fliplr(0.5)
            # TODO add augmentations here
        ])

        #TODO use parameter images= instead of image=
        image_aug, psoi_aug = aug(image=img, polygons=_polygons_on_image)
        # image_aug, psoi_aug = aug(image=image, polygons=_polygons_on_image)
        # remove polygons that are outside of the image
        clipped_psoi_aug = psoi_aug.remove_out_of_image(fully=True, partly=False).clip_out_of_image()
        image_poly_aug = clipped_psoi_aug.draw_on_image(image_aug, alpha_face=0.2, size_points=10)
        images_poly_aug.append(image_poly_aug)
        
        annos_list = []
        for _pol in psoi_aug:
            x1, y1 = _pol[0]
            x2, y2 = _pol[1]
            x3, y3 = _pol[2]
            x4, y4 = _pol[3]
            class_id = 0
            #TODO only working for one label right now
            cat = 0
            coordinates = [x1, y1, x2, y2, x3, y3, x4, y4]
            labels = [cat, *coordinates]
            assert len(coordinates) == 8, "Label does not have four points"
            labels = [cat, *coordinates]
            augImageHeight, augImageWidth, _ = image_aug.shape
            label, label_pixel = normalize_anchors(labels, augImageHeight, augImageWidth)
            annos_list.append(label)
        annos_array = np.array(annos_list)
        imageio.imwrite(aug_image_path)
        # write to new_json_path
        np.savetxt(str(aug_label_path), annos_array, fmt=["%i"]+["%.6f"]*8)


augment_data()

AssertionError: (1942, 2590, 3)